In [1]:
import os
from dataclasses import dataclass
from typing import List, Tuple

import numpy as np
import pandas as pd
import databento as db
import statsmodels.api as sm
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["axes.grid"] = True

/Library/Python/3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# ========= CONFIG =========

@dataclass
class ProjectConfig:
    dataset: str = "GLBX.MDP3"
    continuous_symbol: str = "ES.c.0"  # front ES continuous future on CME Globex
    start_date: str = "2019-01-01"
    end_date: str = "2024-12-31"
    event_window_k: int = 5          # +/- k days around rebalance
    strat_k_entry: int = 3           # enter k days before rebalance
    contract_multiplier: int = 50    # ES multiplier: $50 per index point
    rebalance_csv_path: str = "rebalance_events.csv"  # you create this

CFG = ProjectConfig()

In [3]:
# ========= DATABENTO CLIENT =========

def get_db_client() -> db.Historical:
    """
    Create a Databento Historical client.
    Uses DATABENTO_API_KEY from environment if present.
    """
    key = "db-YAMakFGLD6VHUbUnFDmD4AKYSyAch"
    if key:
        return db.Historical(key)
    return db.Historical()

client = get_db_client()

In [4]:
def get_es_continuous_ohlcv(
    start: str,
    end: str,
    dataset: str = CFG.dataset,
    continuous_symbol: str = CFG.continuous_symbol,
) -> pd.DataFrame:
    """
    Fetch daily OHLCV for ES continuous front-month future from Databento.
    Uses ohlcv-1d schema and 'continuous' symbology.
    Returns a DataFrame indexed by date (naive), with columns: open, high, low, close, volume.
    """
    data = client.timeseries.get_range(
        dataset=dataset,
        schema="ohlcv-1d",
        symbols=continuous_symbol,
        stype_in="continuous",
        start=start,
        end=end,
    )
    df = data.to_df().copy()

    # Ensure datetime index and then convert to naive date index
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)
    if df.index.tz is not None:
        df.index = df.index.tz_convert("UTC").tz_localize(None)

    # If Databento gives one row per day, this is enough; if multiple, keep last per calendar day.
    df["date"] = df.index.date
    df = (
        df.groupby("date")
        .agg({"open": "first", "high": "max", "low": "min", "close": "last", "volume": "sum"})
        .rename_axis("date")
    )

    return df[["open", "high", "low", "close", "volume"]]


es_df = get_es_continuous_ohlcv(CFG.start_date, CFG.end_date)
es_df.head()


/var/folders/p2/09s9fm497ql1r7hkb3d2cbwc0000gp/T/ipykernel_66574/232786595.py:12: BentoWarning: The streaming request contained one or more days which have reduced quality: 2019-01-15 (degraded), 2019-02-22 (degraded), 2019-03-13 (degraded)... See: https://databento.com/docs/api-reference-historical/metadata/metadata-get-dataset-condition
  data = client.timeseries.get_range(


,open,high,low,close,volume
date,,,,,
2019-01-01,2508.00,2521.00,2506.25,2517.00,14934
2019-01-02,2517.25,2521.25,2452.25,2474.75,1933580
2019-01-03,2475.00,2493.50,2443.25,2449.50,2172622
2019-01-04,2449.25,2539.25,2438.50,2529.50,2255998
2019-01-06,2537.25,2538.75,2532.50,2538.25,15496


In [5]:
import pandas as pd
from pandas.tseries.offsets import WeekOfMonth

# ========= 1. Load rebalance events =========

def sp500_quarterly_rebalance_dates(start_year: int, end_year: int) -> pd.DatetimeIndex:
    """
    Approximate S&P 500 scheduled rebalance dates as the
    third Friday of March, June, September, and December
    for each year in [start_year, end_year].

    Returns a DatetimeIndex of those dates.
    """
    dates = []
    for year in range(start_year, end_year + 1):
        for month in [3, 6, 9, 12]:
            # Third Friday:
            #   WeekOfMonth(week=2, weekday=4)
            #   - weekday=4 means Friday (0=Mon,...,4=Fri)
            #   - week=2 means "third" occurrence (0=first,1=second,2=third)
            base = pd.Timestamp(year=year, month=month, day=1)
            d = base + WeekOfMonth(week=2, weekday=4)
            dates.append(d)
    return pd.DatetimeIndex(dates).sort_values()

# Build rebalance dates matching your ES sample
rebalance_dates = sp500_quarterly_rebalance_dates(2019, 2024)

rebalance_dates

DatetimeIndex(['2019-03-15', '2019-06-21', '2019-09-20', '2019-12-20',
               '2020-03-20', '2020-06-19', '2020-09-18', '2020-12-18',
               '2021-03-19', '2021-06-18', '2021-09-17', '2021-12-17',
               '2022-03-18', '2022-06-17', '2022-09-16', '2022-12-16',
               '2023-03-17', '2023-06-16', '2023-09-15', '2023-12-15',
               '2024-03-15', '2024-06-21', '2024-09-20', '2024-12-20'],
              dtype='datetime64[ns]', freq=None)

In [6]:
events_df = pd.DataFrame({
    "event_date": rebalance_dates.date  # store as date, not full timestamp
})

events_df = events_df.sort_values("event_date").reset_index(drop=True)
events_df.head()

,event_date
0,2019-03-15
1,2019-06-21
2,2019-09-20
3,2019-12-20
4,2020-03-20


In [7]:
# ========= 2. Load S&P 500 weights (daily, 2019–2024) =========

weights_df = pd.read_csv("sp500_weights_2019_2024.csv", parse_dates=["date"])
weights_df["date"] = weights_df["date"].dt.date   # make comparable with event_date
weights_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'sp500_weights_2019_2024.csv'

In [8]:
# ========= 3. Compute ΔW (delta_weight) from weights =========

def compute_delta_weight_from_weights(
    weights_daily: pd.DataFrame,
    events: pd.DataFrame,
) -> pd.DataFrame:
    """
    Given daily S&P500 weights and a list of rebalance event_dates,
    compute:
        w_added   = sum of weights of names added between prev and this rebalance
        w_deleted = sum of weights of names deleted
        delta_weight = w_added - w_deleted

    We assume:
       weights_daily columns: ['date', 'permno', 'ticker', 'weight']
       events columns: ['event_date', ...]
    """
    w = weights_daily.copy()
    w["date"] = pd.to_datetime(w["date"]).dt.date

    ev = events.sort_values("event_date").reset_index(drop=True)

    records = []
    for i in range(1, len(ev)):
        prev_date = ev.loc[i - 1, "event_date"]
        this_date = ev.loc[i, "event_date"]

        w_prev = w[w["date"] == prev_date]
        w_this = w[w["date"] == this_date]

        # If either date is missing from the weights (e.g. before 2019), skip
        if w_prev.empty or w_this.empty:
            continue

        prev_ids = set(w_prev["permno"])
        this_ids = set(w_this["permno"])

        added_ids   = this_ids - prev_ids
        deleted_ids = prev_ids - this_ids

        w_added   = w_this.loc[w_this["permno"].isin(added_ids), "weight"].sum()
        w_deleted = w_prev.loc[w_prev["permno"].isin(deleted_ids), "weight"].sum()
        delta_w   = float(w_added - w_deleted)

        records.append(
            {
                "event_date": this_date,
                "w_added": float(w_added),
                "w_deleted": float(w_deleted),
                "delta_weight": delta_w,
            }
        )

    return pd.DataFrame(records)


deltaW_df = compute_delta_weight_from_weights(weights_df, events_df)
deltaW_df.head()

,event_date,w_added,w_deleted,delta_weight
0,2019-06-21,0.003804,0.004545,-0.000742
1,2019-09-20,0.004127,0.004578,-0.000451
2,2019-12-20,0.006074,0.004595,0.001480
3,2020-03-20,0.002111,0.001203,0.000908
4,2020-06-19,0.004184,0.004797,-0.000613


In [9]:
# ========= 4. Merge ΔW into events_df and align with ES sample =========

# merge
events_df = events_df.merge(
    deltaW_df[["event_date", "w_added", "w_deleted", "delta_weight"]],
    on="event_date",
    how="left",
)

# make ES index a DatetimeIndex (if not already) and compute log returns
es_df_ev = es_df.copy()
if not isinstance(es_df_ev.index, pd.DatetimeIndex):
    es_df_ev.index = pd.to_datetime(es_df_ev.index)

es_df_ev = es_df_ev.sort_index()
es_df_ev["log_ret"] = np.log(es_df_ev["close"]).diff()
es_df_ev = es_df_ev.dropna(subset=["log_ret"])

# keep only events that lie inside the ES sample and have delta_weight
min_es_date = es_df_ev.index.min().date()
max_es_date = es_df_ev.index.max().date()

events_df = events_df[
    (events_df["event_date"] >= min_es_date)
    & (events_df["event_date"] <= max_es_date)
]
events_df = events_df.dropna(subset=["delta_weight"]).reset_index(drop=True)

events_df.head()

,event_date,w_added,w_deleted,delta_weight
0,2019-06-21,0.003804,0.004545,-0.000742
1,2019-09-20,0.004127,0.004578,-0.000451
2,2019-12-20,0.006074,0.004595,0.001480
3,2020-03-20,0.002111,0.001203,0.000908
4,2020-06-19,0.004184,0.004797,-0.000613


In [10]:
# ========= 5. Build event windows around each rebalance =========

def build_event_windows(
    es_with_ret: pd.DataFrame,
    events: pd.DataFrame,
    k: int,
) -> pd.DataFrame:
    """
    Build a panel of ES log returns in [-k, +k] trading days around
    each event_date.

    Returns DataFrame with columns:
        event_date, date, rel_day, log_ret
    """
    es = es_with_ret.copy()
    if not isinstance(es.index, pd.DatetimeIndex):
        es.index = pd.to_datetime(es.index)
    es = es.sort_index()

    records = []

    for _, row in events.iterrows():
        ed = pd.to_datetime(row["event_date"])
        if ed not in es.index:
            # if exact date missing (e.g., holiday), skip
            continue

        center_idx = es.index.get_loc(ed)
        # get_loc might return slice if duplicates, but ES daily shouldn't
        if isinstance(center_idx, slice):
            center_idx = center_idx.start

        start_idx = max(center_idx - k, 0)
        end_idx   = min(center_idx + k, len(es) - 1)

        window = es.iloc[start_idx : end_idx + 1].copy()
        rel_days = np.arange(start_idx - center_idx, end_idx - center_idx + 1)

        tmp = pd.DataFrame(
            {
                "event_date": row["event_date"],
                "date": window.index.date,
                "rel_day": rel_days,
                "log_ret": window["log_ret"].values,
            }
        )
        records.append(tmp)

    if records:
        ev_panel = pd.concat(records, ignore_index=True)
    else:
        ev_panel = pd.DataFrame(columns=["event_date", "date", "rel_day", "log_ret"])

    return ev_panel


event_panel = build_event_windows(es_df_ev, events_df, CFG.event_window_k)
event_panel.head()

,event_date,date,rel_day,log_ret
0,2019-06-21,2019-06-16,-5,0.001296
1,2019-06-21,2019-06-17,-4,-0.001210
2,2019-06-21,2019-06-18,-3,0.011090
3,2019-06-21,2019-06-19,-2,0.004265
4,2019-06-21,2019-06-20,-1,0.005687


In [11]:
# ========= 6. Baseline & abnormal returns, CAR =========

def compute_baseline_and_ar(
    es_with_ret: pd.DataFrame,
    event_panel: pd.DataFrame,
    k: int,
):
    """
    Baseline: ES log_return mean & std on days NOT in any event window.
    AR: log_ret - baseline_mean
    CAR: cumulative AR per event over rel_day.
    """
    es = es_with_ret.copy()
    if not isinstance(es.index, pd.DatetimeIndex):
        es.index = pd.to_datetime(es.index)

    window_dates = pd.to_datetime(event_panel["date"].unique())
    mask = es.index.isin(window_dates)
    baseline_ret = es.loc[~mask, "log_ret"]

    mu = baseline_ret.mean()
    sigma = baseline_ret.std(ddof=1)

    ev = event_panel.copy()
    ev["ar"] = ev["log_ret"] - mu
    ev["car"] = ev.groupby("event_date")["ar"].cumsum()

    baseline_stats = {"mu": mu, "sigma": sigma}
    return ev, baseline_stats


event_panel, baseline_stats = compute_baseline_and_ar(
    es_df_ev, event_panel, CFG.event_window_k
)

baseline_stats

{'mu': 0.00046448776843575917, 'sigma': 0.010755261352675833}

In [12]:
# ========= 7. Volatility diagnostics (optional but useful) =========

def compute_vol_volumetrics(
    event_panel: pd.DataFrame,
    baseline_stats: dict,
) -> pd.DataFrame:
    """
    For each event, compute realized volatility of ES log returns
    inside the event window and compare to baseline sigma.
    """
    rows = []
    for ed, grp in event_panel.groupby("event_date"):
        win_vol = grp["log_ret"].std(ddof=1)
        rows.append(
            {
                "event_date": ed,
                "window_vol": win_vol,
                "baseline_vol": baseline_stats["sigma"],
                "vol_ratio": win_vol / baseline_stats["sigma"] if baseline_stats["sigma"] != 0 else np.nan,
            }
        )
    return pd.DataFrame(rows)


vol_stats = compute_vol_volumetrics(event_panel, baseline_stats)
vol_stats.head()


,event_date,window_vol,baseline_vol,vol_ratio
0,2019-06-21,0.005556,0.010755,0.516597
1,2019-09-20,0.004369,0.010755,0.406231
2,2019-12-20,0.002073,0.010755,0.192756
3,2020-03-20,0.053575,0.010755,4.981288
4,2020-06-19,0.023188,0.010755,2.156008


In [13]:
# ========= 8. Simple ES strategy around rebalances (optional) =========

def backtest_rebalance_strategy(
    es_with_ret: pd.DataFrame,
    events: pd.DataFrame,
    k_entry: int,
    contract_mult: int,
) -> pd.DataFrame:
    """
    Very simple strategy:
      - Enter 1 ES contract at close k_entry trading days BEFORE event_date
      - Exit at close on event_date
      - Always long (you can change sign if you want).
    """
    es = es_with_ret.copy()
    if not isinstance(es.index, pd.DatetimeIndex):
        es.index = pd.to_datetime(es.index)
    es = es.sort_index()

    trades = []

    for _, row in events.iterrows():
        ed = pd.to_datetime(row["event_date"])
        if ed not in es.index:
            continue

        center_idx = es.index.get_loc(ed)
        if isinstance(center_idx, slice):
            center_idx = center_idx.start

        entry_idx = center_idx - k_entry
        if entry_idx < 0:
            continue

        entry_date = es.index[entry_idx]
        exit_date  = es.index[center_idx]

        entry_px = es.loc[entry_date, "close"]
        exit_px  = es.loc[exit_date, "close"]

        pnl_points = exit_px - entry_px  # long 1 contract
        pnl_dollars = pnl_points * contract_mult

        trades.append(
            {
                "event_date": row["event_date"],
                "entry_date": entry_date.date(),
                "exit_date": exit_date.date(),
                "entry_px": float(entry_px),
                "exit_px": float(exit_px),
                "pnl_points": float(pnl_points),
                "pnl_dollars": float(pnl_dollars),
            }
        )

    return pd.DataFrame(trades)


trades_df = backtest_rebalance_strategy(
    es_df_ev, events_df, CFG.strat_k_entry, CFG.contract_multiplier
)
trades_df.head()

,event_date,entry_date,exit_date,entry_px,exit_px,pnl_points,pnl_dollars
0,2019-06-21,2019-06-18,2019-06-21,2924.25,2951.50,27.25,1362.5
1,2019-09-20,2019-09-17,2019-09-20,3004.25,3013.75,9.50,475.0
2,2019-12-20,2019-12-17,2019-12-20,3190.75,3219.25,28.50,1425.0
3,2020-03-20,2020-03-17,2020-03-20,2425.25,2445.00,19.75,987.5
4,2020-06-19,2020-06-16,2020-06-19,3135.50,3196.00,60.50,3025.0


In [14]:
# ========= 9. Regression of CAR on ΔW =========

def run_delta_weight_regression(
    event_panel: pd.DataFrame,
    events: pd.DataFrame,
    horizon_k: int,
):
    """
    Regress CAR_{0,+k} on delta_weight.

    Returns (model, regression_df).
    """
    ev = event_panel.copy()

    # take CAR at rel_day = +k (or last available if shorter window)
    target = ev[ev["rel_day"] == horizon_k][["event_date", "car"]].copy()

    df = target.merge(
        events[["event_date", "delta_weight"]],
        on="event_date",
        how="inner",
    ).dropna(subset=["delta_weight", "car"])

    X = sm.add_constant(df["delta_weight"])
    y = df["car"]

    model = sm.OLS(y, X).fit()
    return model, df


reg_model, reg_df = run_delta_weight_regression(
    event_panel, events_df, CFG.event_window_k
)

print(reg_model.summary())
reg_df.head()

                            OLS Regression Results                            
Dep. Variable:                    car   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.039
Method:                 Least Squares   F-statistic:                    0.1820
Date:                Wed, 03 Dec 2025   Prob (F-statistic):              0.674
Time:                        16:54:22   Log-Likelihood:                 48.951
No. Observations:                  23   AIC:                            -93.90
Df Residuals:                      21   BIC:                            -91.63
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0015      0.008      0.200   

,event_date,car,delta_weight
0,2019-06-21,0.009992,-0.000742
1,2019-09-20,-0.014632,-0.000451
2,2019-12-20,0.017104,0.001480
3,2020-03-20,-0.018521,0.000908
4,2020-06-19,0.007086,-0.000613
